Indicar el camino hacia los archivos "projects.xlsx" y "paises.xlsx

In [ ]:
PATH_PROJECTS= "/content/projects.xlsx" #projects.xlsx
PATH_PAISES= "/content/paises.xlsx" #paises.xlsx

# Importando librerías y archivo principal

In [ ]:
import numpy as np
import spacy
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

from nltk.corpus import wordnet as wn
from nltk.util import ngrams
import gensim.corpora as corpora

from gensim.utils import simple_preprocess
from gensim.models import phrases, word2vec
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import inaugural, stopwords
from wordcloud import STOPWORDS,WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Importando el xlsx propocionado ("projects.xlsx")


In [ ]:
df = pd.read_excel(PATH_PROJECTS)


FileNotFoundError: ignored

Importando el xlsx auxiliar (paises2.xlsx)

In [ ]:
iso_codes = pd.read_excel(PATH_PAISES)

#**Preprocesado de textos**#


In [ ]:
df_a_procesar1 = pd.DataFrame(df)
seleccion_columnas = df_a_procesar1[["title","summary"]]
new_df = seleccion_columnas.copy()
new_df["texto_analizar"] = new_df[["summary", "title"]].apply(" ".join, axis=1)
#variable que usaremos en las tareas de regresión
y = df.ecMaxContribution


#**Pipeline**#

In [ ]:
#palabras que se repiten mucho y meten ruido innecesario
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['new','project'])

In [ ]:
def pipeline(text):
    tokenization = word_tokenize(str(text).replace("'", "").lower())
    # Para eliminar todos los tokens alfanuméricos
    punctuaction = [i for i in tokenization if i.isalpha()]
    text = [WordNetLemmatizer().lemmatize(word) for word in punctuaction]
    stop_w = [j for j in text if j not in stop_words]
    return " ".join(stop_w)

In [ ]:
new_df["texto_analizar"] = new_df["texto_analizar"].apply(pipeline)

# Extracción de temáticas LDA

In [ ]:
import gensim

In [ ]:
corpus_clean = new_df["texto_analizar"].map(lambda x: x.split(' '))

Creando un diccionario con la lista de tokens

In [ ]:
D = gensim.corpora.Dictionary(corpus_clean)
n_tokens = len(D)

Convirtiendo una lista de tokens en una bag-of-words

In [ ]:
corpus_bow = [D.doc2bow(doc) for doc in corpus_clean]

## Evaluación del modelo LDA usando la coherencia ##


In [ ]:
from gensim.models import CoherenceModel

Aquí está comentada la función que usamos para poder seleccionar el número de tópicos, la elección se toma en base a la coherencia

In [ ]:
"""def calculate_coherence_score(n, alpha, beta):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_bow,
                                           id2word=D,
                                           num_topics=n,
                                           alpha=alpha,
                                           per_word_topics=True,
                                           eta = beta)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus_clean, dictionary=D, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

#lista de hiperparámetros
n_topics = [40,50,60]
alpha_list = ['symmetric',0.5]
beta_list = ['auto',0.1,0.2]

for n in n_topics:
    for alpha in alpha_list:
        for beta in beta_list:
            coherence_score = calculate_coherence_score(n, alpha, beta)
            print(f"n : {n} ; alpha : {alpha} ; beta : {beta} ; Score : {coherence_score}")
"""

In [ ]:
num_topics = 60
beta = 0.2
ldag = gensim.models.ldamodel.LdaModel(corpus=corpus_bow, id2word=D, num_topics=num_topics,eta = beta, alpha='symmetric')

#Representación de tópicos#

In [ ]:
def doc_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = ldag.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

new_df['lda_features'] = list(map(lambda doc:doc_to_lda_features(ldag, doc),corpus_bow))

In [ ]:
# Representación de los tópicos más relevantes
ldag.print_topics(num_topics=-1, num_words=10)

#Búsqueda de hiperparámetros para los modelos de Regresión#

In [ ]:
#parameters = {'n_estimators':[500,600,700,1000],'learning_rate':[0.1,0.01],'max_depth':[5,7],'gamma':[0.01]}
#gsearch = GridSearchCV(estimator=xgb_cv, param_grid = parameters ,cv=5)


# Análisis tarea regresión TFIDF#

In [ ]:
#convertimos un conjunto de documentos en una matriz términos-frecuencia-inversa
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(new_df["texto_analizar"])

In [ ]:
xgb_cv  = XGBRegressor(n_estimator=500,objective ='reg:squarederror')
record_scores = cross_val_score(xgb_cv, X_tfidf, y,cv=5)
print("Scores:", record_scores)
print(f'Mean accuracy: {np.mean(record_scores):.4f} (std: +/- {np.std(record_scores):.4f})')

#Análisis tarea regresión LDA#

In [ ]:
X_lda = np.array(list(map(np.array, new_df['lda_features'])))

In [ ]:
xgb_cv  = XGBRegressor(n_estimator=1000)
record_scores = cross_val_score(xgb_cv, X_lda, y,cv=5)
print("Scores:", record_scores)
print(f'Mean accuracy: {np.mean(record_scores):.4f} (std: +/- {np.std(record_scores):.4f})')

#Tarea de regresión LDA añadiendo metadatos#

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder(categories='auto')
x_oh = onehotencoder.fit_transform(df.fundingScheme.values.reshape(-1,1)).toarray()

In [ ]:
df_a_procesar = pd.DataFrame(df)
seleccion_columnas = df_a_procesar[["title","summary"]]
new_df_meta = seleccion_columnas.copy()
new_df_meta["texto_analizar"] = new_df_meta[["title","summary"]].apply(" ".join, axis=1)
new_df_meta

In [ ]:
new_df_meta["texto_analizar"] = new_df_meta["texto_analizar"].apply(pipeline)

In [ ]:
corpus_clean_meta = new_df_meta["texto_analizar"].map(lambda x: x.split(' '))

In [ ]:
D_meta = gensim.corpora.Dictionary(corpus_clean_meta)
n_tokens = len(D_meta)

In [ ]:
corpus_bow_meta = [D.doc2bow(doc) for doc in corpus_clean_meta]

In [ ]:
num_topics = 60
ldag = gensim.models.ldamodel.LdaModel(corpus=corpus_bow_meta, id2word=D_meta, num_topics=num_topics)

In [ ]:
new_df_meta['lda_features'] = list(map(lambda doc:doc_to_lda_features(ldag, doc),corpus_bow_meta))

In [ ]:
X_lda = np.array(list(map(np.array, new_df['lda_features'])))

In [ ]:
X_meta_LDA = np.concatenate([X_lda,x_oh], axis=1)

In [ ]:
xgb_meta_LDA  = XGBRegressor(n_estimator=500)
record_scores = cross_val_score(xgb_meta_LDA,X_meta_LDA,y,cv=5)
print("Scores:", record_scores)
print(f'Mean accuracy: {np.mean(record_scores):.4f} (std: +/- {np.std(record_scores):.4f})')

#Tarea de regresión TF-IDF añadiendo metadatos#

In [ ]:
df_a_procesar = pd.DataFrame(df)
seleccion_columnas = df_a_procesar[["title","summary"]]
new_df_meta_TF = seleccion_columnas.copy()
new_df_meta_TF["text"] = new_df_meta_TF[["title","summary"]].apply(" ".join, axis=1)

In [ ]:
new_df_meta_TF['lda_features'] = list(map(lambda doc:doc_to_lda_features(ldag, doc),corpus_bow_meta))

In [ ]:
X_TF = np.array(list(map(np.array, new_df['lda_features'])))

In [ ]:
vectorizer = TfidfVectorizer()
X_tfidf_meta = vectorizer.fit_transform(new_df_meta_TF["text"])
y = df.ecMaxContribution

In [ ]:
X_meta_TF = hstack([X_tfidf_meta, x_oh])

In [ ]:
xgb_cv  = XGBRegressor(n_estimator=500,objective ='reg:squarederror')
record_scores = cross_val_score(xgb_cv, X_meta_TF, y,cv=5)
print("Scores:", record_scores)
print(f'Mean accuracy: {np.mean(record_scores):.4f} (std: +/- {np.std(record_scores):.4f})')

#Extension: Usando bigramas en el algortimo LDA#

In [ ]:
# Construyendo bigramas
bigram = gensim.models.Phrases(new_df["texto_analizar"], min_count=5, threshold=100) # higher threshold fewer phrases.

In [ ]:
corpus_clean = new_df["texto_analizar"].map(lambda x: x.split(' '))

In [ ]:
# Forma alternativa y más rápida de crear bigramas
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [ ]:
# Define functions for bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_bigrams = make_bigrams(corpus_clean)

In [ ]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
# Creando el diccionario
id2word = corpora.Dictionary(data_lemmatized)

# creando el corpus
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=60,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
new_df["bigramas"]=data_words_bigrams

#Modelo de regresión usando bigramas#


In [ ]:
def doc_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = lda_model.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
new_df['bigramas'] = list(map(lambda doc:doc_to_lda_features(lda_model, doc),corpus))

In [ ]:
X_lda = np.array(list(map(np.array, new_df['bigramas'])))
y = df.ecMaxContribution

In [ ]:
xgb_cv  = XGBRegressor(n_estimator=500)
record_scores = cross_val_score(xgb_cv, X_lda, y,cv=5)
print("Scores:", record_scores)
print(f'Mean accuracy: {np.mean(record_scores):.4f} (std: +/- {np.std(record_scores):.4f})')

# DASHBOARD

## Instalación e importación de paquetes

In [ ]:
!pip install jupyter-dash -q
!pip install dash-cytoscape -q
!pip install pycountry

In [ ]:
from jupyter_dash import JupyterDash  # pip install dash
import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import pandas as pd  # pip install pandas
import plotly.express as px
import plotly.graph_objects as go
import math
from dash import no_update

## Creación de nuesvos dataframes
Creación de dataframes auxiliares que eliminen campos que no vamos a usar para que sea más rápido

In [ ]:
new_df2 = new_df
new_df2['lda_features_max'] = new_df2['lda_features'].apply(np.argmax)
 #Añado fechas de inicio y final
new_df2['startDate'] = df_a_procesar1['startDate']
new_df2['endDate'] = df_a_procesar1['endDate']
#Añado el pais coordinador
new_df2['coordinatorCountry'] = df_a_procesar1['coordinatorCountry']
new_df2['coordinatorCountry'] = new_df2['coordinatorCountry'].str.split(';').str[0].dropna()

# Cargamos el archivo de correspondencias que asocia códigos ISO de dos letras con códigos ISO de tres letras
iso_codes = iso_codes[["name","iso2","iso3"]]


counts_lda_features_max = new_df2['lda_features_max'].value_counts()
df_lda_features_max = new_df2[["lda_features_max", "coordinatorCountry"]]
df_lda_features_max = df_lda_features_max.merge(iso_codes, left_on='coordinatorCountry', right_on='iso2')
df_lda_features_max = df_lda_features_max[["lda_features_max","name", "iso3"]]


counts_coordinatorCountry = new_df2['coordinatorCountry'].value_counts()
#df_counts_lda = pd.DataFrame({'lda_features_max': counts_lda_features_max.index, 'count':counts_lda_features_max.values})
df_counts_lda=new_df2
df_counts_lda['startDate'] = pd.to_datetime(new_df2['startDate'])
df_counts_lda['startDateY']=df_counts_lda['startDate'].dt.strftime('%Y')
df_counts_lda['startDateM']=df_counts_lda['startDate'].dt.strftime('%M')
df_counts_lda['startDateD']=df_counts_lda['startDate'].dt.strftime('%D')

df_counts_lda['endDate'] = pd.to_datetime(new_df2['endDate'])
df_counts_lda['endDateY']=df_counts_lda['endDate'].dt.strftime('%Y')
df_counts_lda['endDateM']=df_counts_lda['endDate'].dt.strftime('%M')
df_counts_lda['sendDateD']=df_counts_lda['endDate'].dt.strftime('%D')

counts_3 = df_counts_lda.groupby(['startDateY', 'lda_features_max']).size().reset_index(name='count')

## Creación de figuras


Gráfico de barras que muestra el tópico mayoritario

In [ ]:
fig_LDA_features = go.Figure(data=[go.Bar(x=counts_lda_features_max.index, y=counts_lda_features_max.values)])
fig_LDA_features.update_layout(xaxis_title='Tópico LDA mayoritario', yaxis_title='Frecuencias', title='Histograma de tópicos LDA mayoritarios')

Gráfico de barras que muestra el tópico mayoritario diferenciando por años

In [ ]:
fig_LDA_features_year = px.bar(counts_3, x="lda_features_max", y="count", color="startDateY")
fig_LDA_features_year.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'}, xaxis_title='Main LDA topic', yaxis_title='Frequency', legend_title="Start year", title ="Main LDA features")
fig_LDA_features_year.update_layout(hovermode="closest")
#hovertemp= "<b>Start Year: </b><br>"
hovertemp = "LDA feature: %{x} <br>"
hovertemp += "Counts: %{y}"
fig_LDA_features_year.update_traces(hovertemplate=hovertemp)
fig_LDA_features_year.show()

Histograma que muestra el número de veces que un pais es coordinador

In [ ]:
fig_coordinatorCountry_hist = go.Figure(data=[go.Bar(x=counts_coordinatorCountry.index, y=counts_coordinatorCountry.values)])
fig_coordinatorCountry_hist.update_layout(xaxis_title='Country', yaxis_title='Coordinated projects', title= 'Coordinator country')

Mapa que muestra los proyectos que han sido coordinados por pais

In [ ]:

# Contamos el número de veces que se repite cada código ISO en el campo 'coordinatorCountry'
counts = new_df2['coordinatorCountry'].value_counts()

# Creamos un dataframe con los resultados de la contabilización
df_coordinatorCountry = pd.DataFrame({'iso_code': counts.index, 'count': counts.values})

# Añadimos a nuestro dataframe la columna 'iso_code_3' que contiene los códigos ISO de tres letras de Plotly
df_coordinatorCountry = df_coordinatorCountry.merge(iso_codes, left_on='iso_code', right_on='iso2')
df_coordinatorCountry = df_coordinatorCountry[["name","iso3", "count"]]

fig_coordinatorCountry_map = go.Figure(
    data = {
        'type':'choropleth',
        'locations':df_coordinatorCountry['name'],
        'locationmode':'country names',
        'colorscale':'Portland',
        'z':df_coordinatorCountry['count'],
        'colorbar':{'title':'Coordinated projects'},
        'marker': {
            'line': {
                'color':'rgb(255,255,255)',
                'width':2
            }
        }

    },
    layout = {
      'geo':{
          'projection':{
              'scale':5,
              'rotation':{
                  'lat':45,
                  'lon':15
              },
              #'type':'bromley'  # default is 'equirectangular'
              #'type':'cylindrical stereographic'  # default is 'equirectangular'
              #'type':'equirectangular'  # default is 'equirectangular'
              #'type':'orthographic'  # default is 'equirectangular'
              'type':'natural earth'  # default is 'equirectangular'


          },
          'scope':'world',
          'showcountries': True,
          'showcoastlines': True
      }
    })
fig_coordinatorCountry_map

In [ ]:
fig_coordinatorCountry_hist_2 = go.Figure(data=[go.Bar(x=counts_coordinatorCountry.index, y=counts_coordinatorCountry.values)])

In [ ]:
import random

topico=40
v0=ldag.print_topics(num_topics=-1,num_words=10)[topico][1]
aux = v0.split('+')
words=[]
prob=[]
for token in aux:
  words.append(token.split('*'))
df_words=pd.DataFrame(words)
df_words=df_words.rename(columns={0: 'prob', 1:'word'})
df_words['prob']=df_words['prob']
df_words['prob']=df_words['prob'].astype(float)*1000
df_words['word']=df_words['word'].str.replace('"','')


colors = [px.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(30)]
data = go.Scatter(x=[random.random() for i in range(100)],
                  y=random.choices(range(100), k=10),
                 mode='text',
                 text=df_words['word'],
                 marker={'opacity':0.45},
                 textfont={'size': df_words['prob']*2,
                           'color': colors},
                 hoverinfo='none')
layout = go.Layout({'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                    'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}}
                   )
nube_palabras = go.Figure(data=[data], layout=layout)

nube_palabras.show()

Aplicación de DASH

In [ ]:
unique_values_hist = df_lda_features_max['lda_features_max'].unique()
unique_values_hist.sort()
options = [{'label':'Feature '+ str(value), 'value': value} for value in unique_values_hist]


app = JupyterDash(__name__)

app.layout = html.Div(children=[
   html.Div(children=[
     html.H1(children= "DASHBOARD", style={
         'textAlign': 'center'
     }),
     html.H5(children= 'Carlos González Gamella: 100364132'),
     html.H5(children='Andrés Grijalba Peña: 100383705')
   ]),
   # elements from the top of the page
   html.Div([html.H1(children='Main LDA feature per year'),
             html.H4("Select/deselect a year. You can double click on a year to show just that year"),
   #html.Div(children='''Dash: First graph.'''),
   dcc.Graph(id="lDA_year", figure=fig_LDA_features_year)
   ]),

  html.H1("Map and chart showing LDA main features"),
  html.H4("Click on a country to show main LDA features for project coordinated that were coordinated by that country. You can scroll through the glove"),
      html.Div(style={'display': 'flex', 'justify-content': 'space-between'}, children=[
        dcc.Graph(id="bar-chart"),
        dcc.Graph(id="mapa", figure=fig_coordinatorCountry_map)
      ]),






   # New Div for all elements in the new 'row' of the page
   html.Div(className='row', children=[
    html.H1("Coordinator country by main LDA topic"),
    html.H4("Select o search a country from the dorpdown list"),
    dcc.Dropdown(id='dropdown', options=options, value=unique_values_hist[0], searchable=True),
    html.Div(style={'display': 'flex', 'justify-content': 'space-between'}, children=[
      dcc.Graph(id='histograma', figure=fig_coordinatorCountry_hist_2, style={'width': '100%'}),
      dcc.Graph(id='nube_palabras', figure=nube_palabras, style={'width': '100%'})
      ])
    ]),
])


@app.callback(Output("bar-chart", "figure"), [Input("mapa", "clickData")])
def update_chart(click_data):
    # Si no se ha hecho clic en el mapa, devuelve un gráfico vacío
    if click_data is None or "points" not in click_data:
        df_filtered = df_lda_features_max
    else:
      country_code = click_data["points"][0]["location"]
      df_filtered = df_lda_features_max[df_lda_features_max["name"] == country_code]
    # Cuenta las veces que se repite lda_features_max en el dataframe filtrado
    print(df_filtered)
    counts = df_filtered["lda_features_max"].value_counts()
    # Devuelve el gráfico de barras con los datos y la configuración
    return {
        "data": [{"x": counts.index, "y": counts.values, "type": "bar"}],
        "layout": {"title": f"LDA features for {country_code or 'every country'}"}
    }

@app.callback(
    Output(component_id='histograma', component_property='figure'),
    Output(component_id='nube_palabras', component_property='figure'),
    Input(component_id='dropdown', component_property='value')
)
def update_figure(selected_value):
    #fig_coordinatorCountry_map2 = actualiza_mapa(selected_value)

  filtered_df = df_lda_features_max[df_lda_features_max['lda_features_max'] == selected_value]
  counts_coordinatorCountry_2 = filtered_df['iso3'].value_counts()
  fig_coordinatorCountry_hist_2 = go.Figure(data=[go.Bar(x=counts_coordinatorCountry_2.index, y=counts_coordinatorCountry_2.values)])
  fig_coordinatorCountry_hist_2.update_layout(xaxis_title='Países', yaxis_title='Frecuencias', title= 'Pais coordinador')

  v0=ldag.print_topics(num_topics=-1)[selected_value][1]
  aux = v0.split('+')
  words=[]
  prob=[]
  for token in aux:
    words.append(token.split('*'))
  df_words=pd.DataFrame(words)
  df_words=df_words.rename(columns={0: 'prob', 1:'word'})
  df_words['prob']=df_words['prob']
  df_words['prob']=df_words['prob'].astype(float)*1000
  df_words['word']=df_words['word'].str.replace('"','')


  colors = [px.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(30)]
  data = go.Scatter(x=[random.random() for i in range(100)],
                  y=random.choices(range(100), k=10),
                  mode='text',
                  text=df_words['word'],
                  marker={'opacity':0.45},
                  textfont={'size': df_words['prob']*1.25,
                            'color': colors},
                  hoverinfo='none')
  layout = go.Layout({'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                      'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}}
                    )
  nube_palabras = go.Figure(data=[data], layout=layout)



  return fig_coordinatorCountry_hist_2, nube_palabras

app.run_server(debug=True, mode='inline')